In [ ]:
import random
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import Workbook
from openpyxl.styles import Alignment, Border, Side
from copy import deepcopy
import re

In [ ]:

# -------------------------
# ПАРАМЕТРЫ
# -------------------------
NUM_STOPS = 13          # Остановки: 0..12
INTERVAL = 5            # Интервал слотов в минутах
SLOTS_PER_DAY = (24*60)//INTERVAL  # 288
DAYS_PER_WEEK = 7
TOTAL_SLOTS = SLOTS_PER_DAY * DAYS_PER_WEEK  # 2016

NUM_BUSES = 8
POPULATION_SIZE = 100
NUM_GENERATIONS = 1000
CROSSOVER_RATE = 0.7
MUTATION_RATE = 0.2
ELITE_SIZE = 3
MAX_DRIVERS = 300

DRIVER_COST = 200.0       # штраф за каждого уникального водителя
NEW_DRIVER_EXTRA_COST = 300.0  # штраф за "добавленного" водителя при разбиении смен

X = 1.0
CYCLE_12 = 4              # логика "2 через 2" для 12h водителей

REQUIRED_COVERAGE = {
    (6, 9): 3.0,
    (9, 17): 1.5,
    (17, 20): 3.0,
    (20, 22): 1.5,
    (22, 24): 1.0,
    (0, 6): 1.0
}

# 8-часовые / 12-часовые
HOURS_8 = 8
HOURS_12 = 12

FORWARD_SEGMENTS  = [(i, i+1) for i in range(NUM_STOPS-1)]
BACKWARD_SEGMENTS = [(i, i-1) for i in range(NUM_STOPS-1,0,-1)]
GLOBAL_NEW_DRIVER_ID = 10000

In [ ]:
def get_required_factor(hour: int) -> float:
    for (start_h, end_h), factor in REQUIRED_COVERAGE.items():
        if start_h <= hour < end_h:
            return factor
    return 1.0

In [ ]:
def create_empty_schedule():
    """Пустой расписание: [bus_idx][slot_idx], каждая ячейка - словарь."""
    schedule = []
    for _ in range(NUM_BUSES):
        bus_row = []
        for _ in range(TOTAL_SLOTS):
            bus_row.append({
                'driver_ids': [],
                'driver_types': [],
                'route_segment': [],
                'is_break': False,
                'driver_home': []
            })
        schedule.append(bus_row)
    return schedule

In [ ]:
def random_initial_solution():
    """
    Генерация расписания:
    - Каждый водитель ездит челноком 0->12->0->12..., с вероятностью ~10% вставляем блок перерыва.
    - Явно записываем в schedule перерывы: driver_ids, route_segment=None, is_break=True.
    - Автобус при этом занят этим водителем (но если хотим "освободить" автобус, можно driver_ids=[],
      тогда вообще в этом слоте автобуса нет).
    """
    schedule = create_empty_schedule()
    driver_pool = []
    driver_id_counter = 0

    for _ in range(NUM_BUSES*3):
        home = random.choice(['0','12'])
        d_type = random.choice(['8h','12h'])
        driver_pool.append({
            'driver_id': driver_id_counter,
            'driver_type': d_type,
            'home_stop': home,
            'availability': [True]*TOTAL_SLOTS
        })
        driver_id_counter+=1

    for bus_idx in range(NUM_BUSES):
        slot_idx=0
        while slot_idx< TOTAL_SLOTS:
            day_of_week = slot_idx//SLOTS_PER_DAY
            time_in_day= slot_idx%SLOTS_PER_DAY
            hour= time_in_day//(60//INTERVAL)
            possible_types = []
            # 12h - "2 через 2"
            if (day_of_week % CYCLE_12)<2:
                possible_types.append('12h')
            # 8h - будни 5..22
            if day_of_week<5 and 5<=hour<22:
                possible_types.append('8h')
            if not possible_types:
                slot_idx+=1
                continue

            chosen_type= random.choice(possible_types)
            shift_len= (HOURS_8 if chosen_type=='8h' else HOURS_12)*(60//INTERVAL)
            if slot_idx+shift_len> TOTAL_SLOTS:
                slot_idx+=1
                continue

            assigned=False
            random.shuffle(driver_pool)
            for drv in driver_pool:
                if drv['driver_type']==chosen_type:
                    # Проверим доступность
                    if all(drv['availability'][s] for s in range(slot_idx, slot_idx+shift_len)):
                        forward= FORWARD_SEGMENTS if drv['home_stop']=='0' else BACKWARD_SEGMENTS
                        backward= BACKWARD_SEGMENTS if drv['home_stop']=='0' else FORWARD_SEGMENTS

                        current_slot= slot_idx
                        used=0
                        direction= forward
                        while used< shift_len:
                            # ~10% вставляем перерыв (6..12 слотов)
                            if random.random()<0.1:
                                break_len= 12 if chosen_type=='8h' else 6
                                if used+break_len<= shift_len:
                                    for _ in range(break_len):
                                        schedule[bus_idx][current_slot]['driver_ids'].append(drv['driver_id'])
                                        schedule[bus_idx][current_slot]['driver_types'].append(chosen_type)
                                        schedule[bus_idx][current_slot]['driver_home'].append(drv['home_stop'])
                                        schedule[bus_idx][current_slot]['route_segment'].append(None)
                                        schedule[bus_idx][current_slot]['is_break']=True
                                        drv['availability'][current_slot]=False
                                        current_slot+=1
                                        used+=1
                                else:
                                    pass
                                continue

                            for seg in direction:
                                if current_slot>=TOTAL_SLOTS or used>=shift_len:
                                    break
                                schedule[bus_idx][current_slot]['driver_ids'].append(drv['driver_id'])
                                schedule[bus_idx][current_slot]['driver_types'].append(chosen_type)
                                schedule[bus_idx][current_slot]['driver_home'].append(drv['home_stop'])
                                schedule[bus_idx][current_slot]['route_segment'].append(seg)
                                schedule[bus_idx][current_slot]['is_break']=False
                                drv['availability'][current_slot]=False
                                current_slot+=1
                                used+=1
                            direction= backward if direction==forward else forward
                        assigned=True
                        break
            if assigned:
                slot_idx+= shift_len
            else:
                if driver_id_counter<MAX_DRIVERS:
                    home = random.choice(['0','12'])
                    new_drv={
                        'driver_id': driver_id_counter,
                        'driver_type': chosen_type,
                        'home_stop': home,
                        'availability':[True]*TOTAL_SLOTS
                    }
                    driver_pool.append(new_drv)
                    driver_id_counter+=1
                slot_idx+=1
    return schedule

In [ ]:
def compute_fitness(schedule):
    """
    Улучшенная логика фитнеса:
    - Сильные штрафы за отсутствие перерывов (8h=1, 12h=2), неправильный размер перерыва,
    - Переработка (8h>96 рабочих слотов, 12h>144 слотов),
    - Недопокрытие в пик,
    - Конфликты, двойные назначения,
    - Начало/конец автобуса не совпадает со start/stop,
    - Завершение смены водителя не на home_stop -> штраф.
    """
    penalty=0.0
    unique_drivers=set()
    coverage=[0]*TOTAL_SLOTS
    driver_slots={}
    bus_end_stop=[[None]*TOTAL_SLOTS for _ in range(NUM_BUSES)]

    for bus_idx in range(NUM_BUSES):
        current_stop=None
        for slot_idx in range(TOTAL_SLOTS):
            cell=schedule[bus_idx][slot_idx]
            d_ids= cell['driver_ids']
            segs= cell['route_segment']
            if len(d_ids)>1:
                penalty+=100.0*(len(d_ids)-1)
            if not d_ids:
                bus_end_stop[bus_idx][slot_idx]= current_stop
                continue

            for i,d_id in enumerate(d_ids):
                unique_drivers.add(d_id)
                if segs[i] is not None:
                    coverage[slot_idx]+=1

                if d_id not in driver_slots:
                    driver_slots[d_id]=[]
                driver_slots[d_id].append({
                    'slot_idx': slot_idx,
                    'bus_idx': bus_idx,
                    'route_segment': segs[i],
                    'is_break': (segs[i] is None),
                    'home_stop': cell['driver_home'][i],
                    'driver_type': cell['driver_types'][i]
                })

            seg= segs[0]
            if seg is None:
                # автобус стоит на месте
                bus_end_stop[bus_idx][slot_idx]= current_stop
            else:
                start_stop,end_stop= seg
                if current_stop is not None and start_stop!=current_stop:
                    penalty+=50.0
                current_stop= end_stop
                bus_end_stop[bus_idx][slot_idx]= end_stop

    # Недопокрытие
    for slot_idx in range(TOTAL_SLOTS):
        day_of_week= slot_idx//SLOTS_PER_DAY
        time_in_day= slot_idx%SLOTS_PER_DAY
        hour= time_in_day//(60//INTERVAL)
        factor= get_required_factor(hour)
        needed= factor*X
        if coverage[slot_idx]< needed:
            penalty+= (needed-coverage[slot_idx])*20.0

    # Двойное назначение (один водитель в разных автобусах на один слот)
    per_driver_time={}
    for d_id,recs in driver_slots.items():
        per_driver_time[d_id]= set()
        for r in recs:
            s_idx= r['slot_idx']
            if s_idx in per_driver_time[d_id]:
                penalty+=80.0
            else:
                per_driver_time[d_id].add(s_idx)

    # Анализ перерывов, переработок и home_stop
    penalty_break_excess=0.0
    penalty_no_break=0.0
    penalty_home=0.0
    penalty_overwork=0.0

    for d_id,recs in driver_slots.items():
        sorted_recs= sorted(recs,key=lambda x:x['slot_idx'])
        d_type= sorted_recs[0]['driver_type']
        home = sorted_recs[0]['home_stop']
        max_break= 12 if d_type=='8h' else 6
        needed_break_count=1 if d_type=='8h' else 2

        # Подсчёт рабочего времени/перерывов
        slots_work=0
        break_lens=[]
        break_count=0
        current_break_len=0
        prev_break=False
        for r in sorted_recs:
            if r['is_break']:
                if not prev_break:
                    break_count+=1
                    current_break_len=1
                else:
                    current_break_len+=1
            else:
                slots_work+=1
                if prev_break and current_break_len>0:
                    break_lens.append(current_break_len)
                current_break_len=0
            prev_break= r['is_break']
        if prev_break and current_break_len>0:
            break_lens.append(current_break_len)

        # Проверка количества перерывов
        if break_count< needed_break_count:
            penalty_no_break+=100.0*(needed_break_count-break_count)
        elif break_count> needed_break_count:
            penalty_no_break+=50.0*(break_count - needed_break_count)

        # Длина перерывов
        for blen in break_lens:
            if blen>max_break:
                penalty_break_excess += 10.0*(blen-max_break)
            elif blen<max_break:
                penalty_break_excess += 10.0*(max_break - blen)

        # Переработка
        if d_type=='8h':
            if slots_work> 8*12:
                penalty_overwork+=50.0*(slots_work- 96)
        else:
            total_slots= len(sorted_recs)  # work+break
            if total_slots> 12*12:
                penalty_overwork+=50.0*(total_slots-144)

        # home_stop: проверяем физический конец
        last_slot = sorted_recs[-1]['slot_idx']
        bus_idx = sorted_recs[-1]['bus_idx']
        end_stop= bus_end_stop[bus_idx][last_slot]
        if end_stop is not None and str(end_stop)!= str(home):
            penalty_home+=80.0

    penalty+=(penalty_break_excess + penalty_no_break + penalty_home + penalty_overwork)

    # Физическая непрерывность автобуса
    for bus_idx in range(NUM_BUSES):
        for slot_idx in range(TOTAL_SLOTS-1):
            end_stop_cur= bus_end_stop[bus_idx][slot_idx]
            next_cell= schedule[bus_idx][slot_idx+1]
            if not next_cell['driver_ids']:
                continue
            segs= next_cell['route_segment']
            if segs and segs[0] is not None:
                next_start= segs[0][0]
                if end_stop_cur!=None and end_stop_cur!= next_start:
                    penalty+=50.0

    fitness= len(unique_drivers)*DRIVER_COST + penalty
    return fitness

In [ ]:
def initial_population(size):
    return [random_initial_solution() for _ in range(size)]

In [ ]:
def selection(pop, fitnesses, elite_size):
    sorted_pop= sorted(zip(pop,fitnesses), key=lambda x:x[1])
    new_pop= [deepcopy(sp[0]) for sp in sorted_pop[:elite_size]]
    inverted= [1.0/(f+1e-9) for f in fitnesses]
    total_inverted= sum(inverted)
    while len(new_pop)< len(pop):
        pick= random.random()* total_inverted
        current=0
        for chromo,invfit in zip(pop,inverted):
            current+=invfit
            if current>pick:
                new_pop.append(deepcopy(chromo))
                break
    return new_pop

In [ ]:
def crossover(parent1,parent2):
    child= create_empty_schedule()
    for bus_idx in range(NUM_BUSES):
        source= parent1 if random.random()<0.5 else parent2
        for slot_idx in range(TOTAL_SLOTS):
            child[bus_idx][slot_idx] = deepcopy(source[bus_idx][slot_idx])
    return child


In [ ]:
def mutate(chromo, mutation_rate=0.1):
    """
    Максимально подробная логика мутации блоков:
      - Собираем блоки (driver_id, bus_idx, [is_break])
      - Для 1-2 случайных блоков делаем:
        (A) Поменять тип водителя (8h↔12h), пересчитать слоты
        (B) Разбить смену, добавив нового водителя (штраф extra driver)
        (C) Сдвинуть блок во времени (если конфликт - ставим break)
        (D) Добавить/убрать кусок перерыва
    """
    new_chromo= deepcopy(chromo)

    # Собираем блоки
    blocks=[]
    for bus_idx in range(NUM_BUSES):
        slot_idx=0
        while slot_idx<TOTAL_SLOTS:
            cell = new_chromo[bus_idx][slot_idx]
            if not cell['driver_ids']:
                slot_idx+=1
                continue
            d_id= cell['driver_ids'][0]
            d_type= cell['driver_types'][0]
            is_br= (cell['route_segment'][0] is None)
            start_slot= slot_idx
            current= slot_idx+1
            while current<TOTAL_SLOTS:
                c2= new_chromo[bus_idx][current]
                if len(c2['driver_ids'])==1 and c2['driver_ids'][0]==d_id:
                    seg2= c2['route_segment'][0]
                    if (seg2 is None) == is_br:
                        current+=1
                        continue
                break
            blocks.append({
                'driver_id': d_id,
                'driver_type': d_type,
                'bus_idx': bus_idx,
                'start_slot': start_slot,
                'end_slot': current,
                'is_break': is_br
            })
            slot_idx=current

    # Мутируем 1..2 блока случайно
    num_blocks_to_mutate = random.randint(1,2)
    for _ in range(num_blocks_to_mutate):
        if not blocks:
            break
        block= random.choice(blocks)
        blocks.remove(block)  # чтобы не мутировать один и тот же блок

        d_id= block['driver_id']
        d_type= block['driver_type']
        bus_idx= block['bus_idx']
        start_slot= block['start_slot']
        end_slot= block['end_slot']
        block_len= end_slot - start_slot
        is_br= block['is_break']

        if random.random()> mutation_rate:
            continue  # не всегда делаем мутацию

        op = random.random()
        # (A) Смена типа 8h<->12h
        if op<0.2:
            new_type = '8h' if d_type=='12h' else '12h'
            for s in range(start_slot, end_slot):
                c2= new_chromo[bus_idx][s]
                if len(c2['driver_ids'])==1 and c2['driver_ids'][0]==d_id:
                    c2['driver_types'][0] = new_type

        # (B) Разбить смену, добавить нового водителя
        elif op<0.4 and not is_br:
            # Если блок_len >96 (8h) или >144 (12h), делим на 2
            threshold= 96 if d_type=='8h' else 144
            if block_len> threshold:
                mid= (start_slot+ end_slot)//2
                # удаляем водителя из второй половины и вставляем "нового" водителя
                for s in range(mid, end_slot):
                    c2= new_chromo[bus_idx][s]
                    if d_id in c2['driver_ids']:
                        idx= c2['driver_ids'].index(d_id)
                        c2['driver_ids'].pop(idx)
                        c2['driver_types'].pop(idx)
                        c2['driver_home'].pop(idx)
                        c2['route_segment'].pop(idx)
                        if not c2['driver_ids']:
                            c2['is_break']=False
                global GLOBAL_NEW_DRIVER_ID
                new_d_id= GLOBAL_NEW_DRIVER_ID
                GLOBAL_NEW_DRIVER_ID+=1

        # (C) Сдвинуть блок
        elif op<0.6:
            offset= random.randint(-10,10)
            new_start= start_slot+ offset
            new_end= end_slot+ offset
            if 0<= new_start< new_end<= TOTAL_SLOTS:
                # Считываем блок
                block_data=[]
                for s in range(start_slot,end_slot):
                    c2= new_chromo[bus_idx][s]
                    if d_id in c2['driver_ids']:
                        idx= c2['driver_ids'].index(d_id)
                        seg= c2['route_segment'][idx]
                        block_data.append(seg)
                        c2['driver_ids'].pop(idx)
                        c2['driver_types'].pop(idx)
                        c2['driver_home'].pop(idx)
                        c2['route_segment'].pop(idx)
                        if not c2['driver_ids']:
                            c2['is_break']=False

                conflict=False
                for s in range(new_start,new_end):
                    if new_chromo[bus_idx][s]['driver_ids']:
                        conflict=True
                        break
                if conflict:
                    # водитель остаётся в перерыве
                    for s in range(new_start,new_end):
                        c2= new_chromo[bus_idx][s]
                        c2['driver_ids'].append(d_id)
                        c2['driver_types'].append(d_type)
                        c2['driver_home'].append('0')
                        c2['route_segment'].append(None)
                        c2['is_break']=True
                else:
                    idx_data=0
                    for s in range(new_start,new_end):
                        c2= new_chromo[bus_idx][s]
                        c2['driver_ids'].append(d_id)
                        c2['driver_types'].append(d_type)
                        c2['driver_home'].append('0')
                        seg= block_data[idx_data] if idx_data<len(block_data) else None
                        c2['route_segment'].append(seg)
                        c2['is_break']= (seg is None)
                        idx_data+=1

        # (D) Добавить/убрать перерывы
        else:
            if not is_br:
                # добавим перерыв внутри
                mid= (start_slot+end_slot)//2
                break_len= 12 if d_type=='8h' else 6
                br_start= mid
                br_end= min(end_slot, br_start+ break_len)
                for s in range(br_start, br_end):
                    c2= new_chromo[bus_idx][s]
                    # если d_id уже там, превращаем в break
                    if d_id in c2['driver_ids']:
                        idx= c2['driver_ids'].index(d_id)
                        c2['route_segment'][idx]= None
                        c2['is_break']=True
                    else:
                        c2['driver_ids'].append(d_id)
                        c2['driver_types'].append(d_type)
                        c2['driver_home'].append('0')
                        c2['route_segment'].append(None)
                        c2['is_break']=True
            else:
                # Уменьшим перерыв (уберём часть слотов)
                reduce_len= min(4, block_len)
                for s in range(start_slot, start_slot+ reduce_len):
                    c2= new_chromo[bus_idx][s]
                    if d_id in c2['driver_ids']:
                        idx= c2['driver_ids'].index(d_id)
                        c2['driver_ids'].pop(idx)
                        c2['driver_types'].pop(idx)
                        c2['driver_home'].pop(idx)
                        c2['route_segment'].pop(idx)
                        if not c2['driver_ids']:
                            c2['is_break']=False

    return new_chromo

In [ ]:
def genetic_algorithm():
    population= initial_population(POPULATION_SIZE)
    best_solution=None
    best_fitness=float('inf')
    fitness_history=[]

    for gen in range(NUM_GENERATIONS):
        fitnesses=[compute_fitness(chromo) for chromo in population]
        for i,f in enumerate(fitnesses):
            if f<best_fitness:
                best_fitness=f
                best_solution=deepcopy(population[i])
        fitness_history.append(best_fitness)
        print(f"Поколение {gen+1}: Лучший фитнес = {best_fitness:.2f}")

        new_pop= selection(population,fitnesses, ELITE_SIZE)
        next_pop= new_pop[:ELITE_SIZE]
        while len(next_pop)< POPULATION_SIZE:
            p1= random.choice(new_pop)
            p2= random.choice(new_pop)
            if random.random()<CROSSOVER_RATE:
                child= crossover(p1,p2)
            else:
                child= deepcopy(p1)
            child= mutate(child,MUTATION_RATE)
            next_pop.append(child)
        population= next_pop

    return best_solution,best_fitness, fitness_history

In [ ]:
def create_schedule_dataframe(solution):
    driver_info={}
    for bus_idx in range(NUM_BUSES):
        for slot_idx in range(TOTAL_SLOTS):
            cell= solution[bus_idx][slot_idx]
            d_ids= cell['driver_ids']
            segs= cell['route_segment']
            homes=cell['driver_home']
            d_types= cell['driver_types']
            is_br= cell['is_break']
            for i,d_id in enumerate(d_ids):
                if d_id not in driver_info:
                    driver_info[d_id]={
                        'home': homes[i],
                        'driver_type': d_types[i],
                        'shifts': []
                    }
                day_of_week= slot_idx//SLOTS_PER_DAY
                time_in_day= slot_idx%SLOTS_PER_DAY
                hour= time_in_day//(60//INTERVAL)
                minute_5_chunk= time_in_day%(60//INTERVAL)
                minute= minute_5_chunk*INTERVAL
                day_str=["Пн","Вт","Ср","Чт","Пт","Сб","Вс"][day_of_week]
                seg_str = f"{segs[i]}" if segs[i] else "BREAK"
                driver_info[d_id]['shifts'].append({
                    'day': day_str,
                    'slot_idx': slot_idx,
                    'time': f"{hour:02d}:{minute:02d}",
                    'bus': bus_idx,
                    'segment': seg_str,
                    'is_break': (segs[i] is None)
                })

    columns={}
    for d_id,info in driver_info.items():
        colname= f"Водитель {d_id} ({info['driver_type']}, home={info['home']})"
        columns[colname]=[]
        sorted_shifts= sorted(info['shifts'], key=lambda x:x['slot_idx'])
        for s in sorted_shifts:
            rec= f"{s['day']} {s['time']} Bus{s['bus']} {s['segment']} break={s['is_break']}"
            columns[colname].append(rec)
    max_len= max(len(v) for v in columns.values())
    for k in columns:
        columns[k]+=[""]*(max_len-len(columns[k]))
    df= pd.DataFrame(columns)
    return df


In [ ]:
best_sol,best_fit,fitness_history= genetic_algorithm()
print("\n=== ЛУЧШЕЕ РЕШЕНИЕ ===")
print(f"Фитнес (стоимость) = {best_fit:.2f}")
df= create_schedule_dataframe(best_sol)
df.to_excel('schedule.xlsx', index=False)

# График изменения фитнеса
plt.figure(figsize=(8,5))
plt.plot(range(1, len(fitness_history)+1), fitness_history, marker='o')
plt.title("Изменение фитнес-функции по поколениям")
plt.xlabel("Поколение")
plt.ylabel("Фитнес (Стоимость)")
plt.grid(True)
plt.show()


Поколение 1: Лучший фитнес = 508730.00
Поколение 2: Лучший фитнес = 508730.00
Поколение 3: Лучший фитнес = 508730.00
Поколение 4: Лучший фитнес = 508730.00
Поколение 5: Лучший фитнес = 507530.00
Поколение 6: Лучший фитнес = 507530.00
Поколение 7: Лучший фитнес = 507530.00
Поколение 8: Лучший фитнес = 507530.00
Поколение 9: Лучший фитнес = 507530.00
Поколение 10: Лучший фитнес = 507130.00
Поколение 11: Лучший фитнес = 505930.00
Поколение 12: Лучший фитнес = 505930.00
Поколение 13: Лучший фитнес = 505930.00
Поколение 14: Лучший фитнес = 505280.00
Поколение 15: Лучший фитнес = 505280.00
Поколение 16: Лучший фитнес = 505280.00
Поколение 17: Лучший фитнес = 505280.00
Поколение 18: Лучший фитнес = 504060.00
Поколение 19: Лучший фитнес = 504060.00
Поколение 20: Лучший фитнес = 504060.00
Поколение 21: Лучший фитнес = 504060.00
Поколение 22: Лучший фитнес = 502800.00
Поколение 23: Лучший фитнес = 500830.00
Поколение 24: Лучший фитнес = 500830.00
Поколение 25: Лучший фитнес = 500670.00
Поколение

In [ ]:
df = pd.read_excel('/content/schedule.xlsx')

In [ ]:
def process_schedule_to_excel(dataframe, output_file):
    workbook = Workbook()
    workbook.remove(workbook.active)  # Удаляем стандартный лист

    # Лист "Итоги"
    days_of_week = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]

    summary_sheet = workbook.create_sheet(title="Итоги")
    summary_sheet.append(["Общее количество водителей", len(dataframe.columns)])
    summary_sheet.append(["День недели", "Количество водителей"])

    # Инициализация счетчиков дней недели
    day_counts = {day: 0 for day in days_of_week}

    # Обработка данных для подсчета водителей по дням недели
    for driver_column in dataframe.columns:
        driver_data = dataframe[driver_column].tolist()
        days = set()  # Уникальные дни для текущего водителя

        for record in driver_data:
            if pd.isna(record):  # Пропускаем NaN
                continue

            record = str(record).strip()
            parsed = re.match(r"(\w{2}) ", record)  # Извлечение сокращённого дня недели
            if parsed:
                short_day = parsed.group(1)
                full_day_name = {
                    "Пн": "Понедельник",
                    "Вт": "Вторник",
                    "Ср": "Среда",
                    "Чт": "Четверг",
                    "Пт": "Пятница",
                    "Сб": "Суббота",
                    "Вс": "Воскресенье"
                }.get(short_day)

                if full_day_name and full_day_name not in days:
                    day_counts[full_day_name] += 1
                    days.add(full_day_name)

    # Запись итоговых данных на лист "Итоги"
    for day, count in day_counts.items():

        summary_sheet.append([day, count])


    day_columns = ["Начало", "Конец", "Действие", "Автобус"]
    for driver_column in dataframe.columns:
        # Проверяем название столбца
        match = re.match(r"Водитель (\d+) \((\d+h), home=(\d+)\)", driver_column)
        if not match:
            continue

        driver_number, shift_type, home_depot = match.groups()
        sheet = workbook.create_sheet(title=f"Водитель {driver_number}")

        # Заголовки
        sheet.append(["Номер водителя", "Вид смены", "Родное депо"] + day_columns * len(days_of_week))
        sheet.merge_cells(start_row=1, start_column=1, end_row=2, end_column=1)
        sheet.merge_cells(start_row=1, start_column=2, end_row=2, end_column=2)
        sheet.merge_cells(start_row=1, start_column=3, end_row=2, end_column=3)

        for i, day in enumerate(days_of_week):
            col_start = 4 + i * len(day_columns)
            sheet.merge_cells(start_row=1, start_column=col_start, end_row=1, end_column=col_start + 3)
            sheet.cell(row=1, column=col_start).value = day
            for j, header in enumerate(day_columns):
                sheet.cell(row=2, column=col_start + j).value = header

        # Вставляем информацию о водителе
        sheet.append([driver_number, shift_type, home_depot])

        # Обработка записей
        driver_data = dataframe[driver_column].dropna().tolist()
        records_by_day = {day: [] for day in days_of_week}

        for day_name in days_of_week:
            current_start_time = None
            current_action = None
            current_bus = None
            for record in driver_data:
                parsed = re.match(r"(\w{2}) (\d{2}:\d{2}) Bus(\d+)(?: \((\d+), (\d+)\))? (BREAK|break=(\w+))", record)
                if not parsed:
                    continue

                day, time, bus, start_stop, end_stop, action, on_break = parsed.groups()
                start_stop = int(start_stop) if start_stop else None
                end_stop = int(end_stop) if end_stop else None
                is_break = action == "BREAK" or (on_break == "True")

                # Пропускаем записи не для текущего дня
                if day_name != {
                    "Пн": "Понедельник",
                    "Вт": "Вторник",
                    "Ср": "Среда",
                    "Чт": "Четверг",
                    "Пт": "Пятница",
                    "Сб": "Суббота",
                    "Вс": "Воскресенье"
                }.get(day, day):
                    continue

                if is_break:
                    # Объединение перерывов
                    if current_action == "На перерыве" and records_by_day[day_name]:
                        records_by_day[day_name][-1] = (current_start_time, time, "На перерыве", bus)
                    else:
                        current_start_time = time
                        records_by_day[day_name].append((time, time, "На перерыве", bus))
                    current_action = "На перерыве"
                    continue

                # Если маршрут продолжается или начинается
                if end_stop in (0, 12):
                    if current_action and current_action.startswith("Едет до депо"):
                        records_by_day[day_name].append((current_start_time, time, current_action, current_bus))
                    current_action = f"Едет до депо {end_stop}"
                    current_bus = bus
                    current_start_time = time

            # Завершение маршрутов
            if current_action and current_action.startswith("Едет до депо"):
                records_by_day[day_name].append((current_start_time, time, current_action, current_bus))

        # Запись в Excel
        for day, records in records_by_day.items():
            row_idx = 3
            col_start = 4 + days_of_week.index(day) * len(day_columns)
            for start, end, action, bus in records:
                sheet.cell(row=row_idx, column=col_start).value = start
                sheet.cell(row=row_idx, column=col_start + 1).value = end
                sheet.cell(row=row_idx, column=col_start + 2).value = action
                sheet.cell(row=row_idx, column=col_start + 3).value = bus
                row_idx += 1

    workbook.save(output_file)



# Пример использования
output_file_path = "/content/drivers_schedule.xlsx"
process_schedule_to_excel(deepcopy(df), output_file_path)
